# Creating the Database
First let's import Sqlite and the other libraries we will need.

In [9]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3

### Create the Database
Now I  will create the Sqlite database. I use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

In [10]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In Sqlite we need to enable the enforcement of foreign key constraints:

In [11]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

In [12]:
comments = pd.read_csv('comments.csv')

In [13]:
posts = pd.read_csv('posts.csv')

In [14]:
users = pd.read_csv('users.csv')

In [15]:
# Creating Users Table
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
(sqlite3.OperationalError) table "users" already exists
[SQL: CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [16]:
# Create Posts Table
%%sql
CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);


 * sqlite:///chatdata.db
(sqlite3.OperationalError) table "posts" already exists
[SQL: CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [17]:
# Creating Comments Table
%%sql
CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER,
FOREIGN KEY ("UserId") REFERENCES users("UserId"),
FOREIGN KEY ("PostId") REFERENCES posts("PostId")
);

 * sqlite:///chatdata.db
(sqlite3.OperationalError) table "comments" already exists
[SQL: CREATE TABLE "comments" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostId" INTEGER,
"Score" INTEGER,
"Text" TEXT,
"CreationDate" TEXT,
"UserId" INTEGER,
FOREIGN KEY ("UserId") REFERENCES users("UserId"),
FOREIGN KEY ("PostId") REFERENCES posts("PostId")
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [18]:
# Insert data into the new users table
users.to_sql("users", con, if_exists='append', index=False)

18412

In [19]:
# Insert data into the new posts table
# TODO
posts.to_sql('posts', con, if_exists='append', index=False)

42234

In [20]:
# Insert data into the new comments table
# TODO
comments.to_sql('comments', con, if_exists='append', index=False)

50000

How many posts have 0 comments?

In [21]:
sql = """
Select Count(*) As Number_of_posts_without_comments
From posts
Where posts.CommentCount = 0
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result




,Number_of_posts_without_comments
0,21713


How many posts have 1 comment?





In [22]:
sql = """
Select Count(*) As Number_of_posts_with_1_comment
From posts
Where posts.CommentCount = 1
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Number_of_posts_with_1_comment
0,6460


How many posts have 2 or more comments?

In [23]:
sql = """
Select Count(*) As Number_of_posts_with_2_or_more_comments
From posts
Where posts.CommentCount >= 2
"""

result = pd.read_sql(sql, con) # con is the connection to the database
result



,Number_of_posts_with_2_or_more_comments
0,14061


What are the 5 posts with highest viewcount?

In [24]:
sql = """
Select posts.Id, posts.viewcount
From posts
Order By posts.viewcount Desc
Limit 5
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,ViewCount
0,388566,19542
1,394118,16317
2,431370,11723
3,398646,9850
4,434128,6718


What are the top 5 posts with the highest scores?

In [25]:
sql = """
Select posts.Id, posts.score
From posts
Order By posts.score Desc
Limit 5
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,Score
0,431397,101
1,394128,100
2,426878,93
3,388578,80
4,431370,77


What are the 5 most frequent scores on posts?

In [26]:
sql = """
Select posts.score, count(posts.score) AS number_of_posts_with_score
From posts
Group By posts.score
Order By count(posts.score) Desc
Limit 5
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Score,number_of_posts_with_score
0,0,19888
1,1,11867
2,2,5094
3,3,2228
4,4,1059


How many posts have the keyword "data" in their tags?

In [27]:
sql = """
Select Count(*) AS Posts_with_Data
From posts
Where posts.Tags Like '%data%'
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Posts_with_Data
0,2242


What are the 5 most frequent commentcount for posts?

In [28]:
sql = """
Select posts.commentcount, count(posts.commentcount)
From posts
Group By posts.commentcount
Order By count(posts.commentcount) Desc
Limit 5
"""

result = pd.read_sql(sql, con) # con is the connection to the database
result



,CommentCount,count(posts.commentcount)
0,0,21713
1,1,6460
2,2,4966
3,3,3063
4,4,2026


How many posts have an accepted answers?

In [29]:
sql = """
Select count(posts.ID) AS Accepted_Answers
From posts
Where posts.AcceptedAnswerID != 0
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Accepted_Answers
0,5341


What is the average reputation of table users?

In [30]:
sql = """
Select AVG(users.Reputation) AS Average_Reputation
From users
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Average_Reputation
0,312.350912


What are the min and max reputation of users?

In [31]:
sql = """
Select MIN(users.Reputation) AS Min_Rep, MAX(users.Reputation) AS Max_Rep
from users
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Min_Rep,Max_Rep
0,1,228662


What is the length of the body of 5 most viewed posts?

In [32]:
sql = """
Select posts.ViewCount AS Views, LENGTH(posts.body) As Characters
from posts
Order By posts.ViewCount Desc
Limit 5
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Views,Characters
0,19542,2270
1,16317,512
2,11723,811
3,9850,2148
4,6718,1172


How many different locations are there in the users table?

In [33]:
sql = """
SELECT COUNT(DISTINCT users.Location) AS Different_Locations
FROM users
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Different_Locations
0,1900


What are the top 5 locations of users?

In [34]:
sql = """
Select users.Location, Count(users.Id)
From users
Where Location != "None"
Group By Location
Order By Count(users.Id) Desc
Limit 5
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Location,Count(users.Id)
0,Germany,117
1,India,100
2,United States,69
3,"Paris, France",66
4,"London, United Kingdom",63


*Rank* the days of the week from highest to lowest in terms of the volume of ViewCount as a percentage.

In [35]:
sql = """
Select CAST(strftime('%w', CreationDate) AS INTEGER) AS Day, SUM(posts.viewcount) * 100.0 / (SELECT SUM(posts.viewcount) FROM posts) AS Percentage
from posts
Group By Day
Order By Percentage DESC
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Day,Percentage
0,4,16.819476
1,3,16.768689
2,2,16.255034
3,1,15.806727
4,5,13.563061
5,0,11.895609
6,6,8.891404


How many posts have been created by a user that has a filled out the "AboutMe" section?

In [36]:
sql = """
Select COUNT(posts.Id) AS Posts_with_about_me
from posts
JOIN users on posts.OwnerUserId = users.Id
Where users.Aboutme NOT NULL
"""

result = pd.read_sql(sql, con) # con is the connection to the database
result



,Posts_with_about_me
0,17189


Considering only the users with an "AboutMe," how many posts are there per user?

In [37]:
sql = """
Select (COUNT(posts.Id) * 1.0) / COUNT(DISTINCT users.Id) AS "Number_of_posts"
from users
Join posts on posts.OwnerUserId = users.Id
WHERE users.AboutMe IS NOT NULL AND users.Aboutme != ' ' ;
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Number_of_posts
0,5.106655


Not taking into account the commentcount field in the table posts, what are the Top 10 posts in terms of number of comments?


In [38]:
sql = """
Select posts.Id, count(comments.Id) AS number_of_comments
From posts
Join comments on posts.Id = comments.PostId
Group BY posts.Id
Order By number_of_comments Desc
Limit 10
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,number_of_comments
0,386853,66
1,386556,34
2,418910,31
3,395232,31
4,402987,27
5,386075,26
6,394118,24
7,402950,23
8,398828,23
9,396111,22


What are the Top 10 posts which have the highest
cummulative (post score + comment score) score?


In [39]:
sql = """
SELECT posts.Id, SUM(posts.score + comments.score) AS Cumulative_Score
from posts
Join comments ON posts.Id = comments.PostId
Group By posts.Id
Order By Cumulative_Score Desc
Limit 10
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,Cumulative_Score
0,394118,1778
1,394128,1569
2,418910,1094
3,398653,1021
4,388578,941
5,388566,885
6,396818,808
7,418814,621
8,394258,570
9,394439,566


Who are the top 10 users who comment the most?


In [40]:
sql = """
Select users.Id, users.reputation, COUNT(comments.Id) AS Num_Of_Comments
from users
Join comments On users.Id = comments.UserId
Group By users.Id
Order By Num_Of_comments Desc
Limit 10
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,Reputation,Num_Of_Comments
0,919,223056,3301
1,805,228662,1153
2,143489,2890,1024
3,11887,39200,805
4,85665,17391,691
5,164061,13485,540
6,22047,41385,536
7,158565,6482,504
8,7962,8030,492
9,35989,71548,470


Who are the top 10 users who post the most?


In [41]:
sql = """
Select users.Id, Count(posts.Id) AS number_of_posts, users.reputation
from users
Join posts on users.Id = posts.OwnerUserId
Group By users.Id
Order By number_of_posts Desc
Limit 10
"""
result = pd.read_sql(sql, con) # con is the connection to the database
result



,Id,number_of_posts,Reputation
0,204068,637,17404
1,85665,545,17391
2,173082,435,42553
3,11887,435,39200
4,686,386,85077
5,1352,285,59160
6,3382,274,24841
7,7224,233,65999
8,35989,230,71548
9,805,230,228662


# Close SQLite

It is good practise to close all relational databases as soon as you are finished updating them.

In [42]:
con.close()